#### Day07: Lab

This lab focuses on extracting and displaying information from a database

In [2]:
# import libraries
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, and_, or_
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy import func

In [3]:
# Setting things up:
engine = sqlalchemy.create_engine('sqlite:///geog.db')
Base = declarative_base() 

In [4]:
# Schemas
class Region(Base):
  __tablename__ = 'regions'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  departments = relationship("Department", backref = "region")

  def __init__(self, name):
    self.name = name 

  def __repr__(self):
    return "<Region('%s')>" % self.id 

In [5]:
class Department(Base):
  __tablename__ = 'departments'

  id = Column(Integer, primary_key=True)
  deptname = Column(String)
  region_id = Column(Integer, ForeignKey('regions.id')) 
  towns = relationship("Town", backref = "department")

  def __init__(self, deptname):
    self.deptname = deptname 

  def __repr__(self):
    return "<Department('%s')>" % self.id 

In [6]:
class Town(Base):
  __tablename__ = 'towns'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  population = Column(Integer)
  dept_id = Column(Integer, ForeignKey('departments.id'))

  def __init__(self, name, population):
    self.name = name 
    self.population = population

  def __repr__(self):
    return "<Town('%s')>" % (self.name)

In [7]:
#First time create tables
Base.metadata.create_all(engine) 

In [8]:
#Create a session to actually store things in the db
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
# Create regions
reg1 = Region('Region 1')
reg2 = Region('Region 2')
reg3 = Region('Region 3')

In [10]:
# Create departments, nested in regions
dept1 = Department('Department 1')
reg1.departments.append(dept1)

dept2 = Department('Department 2')
reg1.departments.append(dept2)

dept3 = Department('Department 3')
reg3.departments.append(dept3)

dept4 = Department('Department 4')
reg2.departments.append(dept4)


In [11]:
# Create towns, nested in departments
t1 = Town("a", 10000)
t2 = Town("b", 20000)
dept1.towns = [t1, t2]

t3 = Town("c", 30000)
t4 = Town("d", 40000)
dept2.towns = [t3, t4]

t5 = Town("e", 50000)
t6 = Town("f", 60000)
dept3.towns = [t5, t6]

t7 = Town("g", 70000)
t8 = Town("h", 80000)
dept4.towns = [t7, t8]

In [12]:
session.add_all([reg1, reg2, reg3])
session.add_all([dept1, dept2, dept3, dept4])
session.add_all([t1, t2, t3, t4, t5, t6, t7, t8])

session.commit()

1. Print all of the towns, and order them by Town.id in descending order.

In [13]:
for town in session.query(Town).order_by(Town.id.desc()):
  print(town.id, town.name, town.population)

8 f 60000
7 e 50000
6 h 80000
5 g 70000
4 d 40000
3 c 30000
2 b 20000
1 a 10000


2. Display, by department, the cities having more than 50,000 inhabitants.

In [20]:
# YOUR CODE HERE
for town in session.query(Town).join(Department).filter(Town.population > 50000).order_by(Department.id.desc()):
    print(town.department.deptname, town.population)

Department 3 60000
Department 4 70000
Department 4 80000


3. Display the total number of inhabitants per department using only a query. (no lists!)

In [29]:
for dept in session.query(Department.deptname, func.sum(Town.population)).join(Department).group_by(Department.deptname):
    print(dept.deptname)

Department 1
Department 2
Department 3
Department 4


In [23]:
# YOUR CODE HERE
for dept in session.query(Department.deptname, func.sum(Town.population).label('inhab')).join(Department).group_by(Department.deptname):
    print(dept.deptname, dept.inhab)

Department 1 30000
Department 2 70000
Department 3 110000
Department 4 150000


In [16]:
# Copyright (c) 2014 Matt Dickenson
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.